In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import os
from tqdm import tqdm

In [2]:
df_central = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/central points/central_emvec_GPT2_allwords.csv')
df_central = df_central.drop(df_central.columns[0],axis=1)
heads = df_central.columns.values.tolist()

df_central_vec = pd.DataFrame()
for head in heads:
    digit_value = []
    values = df_central[head].tolist()
    for indeics in tqdm(range(len(values))):
        value = values[indeics]
        value = value.strip('[').strip(']').replace('\n','').replace('  ',' ').replace('   ',' ').split(' ')
        value = [x for x in value if len(x) != 0]
        if len(value) != 768:
            value = value[0:768]
        value = [float(x) for x in value]
        digit_value.append(value)
    df_central_vec[head] = digit_value

100%|███████████████████████████████████████| 480/480 [00:00<00:00, 3487.23it/s]


In [3]:
def get_float_GPT2BERT_vector(vectorstrlist):
    float_vectors = []
    for item in vectorstrlist:
        item = item.replace(" ","").replace("\n","")
        item = item.strip("(").strip(")").strip("[").strip("]")
        item = item.split(',')[0:768]
        item[767] = item[767].strip(']')
        item = [float(x) for x in item]
        float_vectors.append(item)
    return float_vectors

In [4]:
df_central_all = pd.read_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/emvecs/emvec_GPT2_allwords.csv')
df_central_all = df_central_all.drop(df_central_all.columns[0],axis=1)
heads = df_central_all.columns.values.tolist()

df_allvec = pd.DataFrame()

for head in heads:
    allvectors = df_central_all[head].tolist()
    length = len(allvectors)
    
    float_vecs = []
    for i in tqdm(range(length)):
        vectors = allvectors[i]
        vectors = vectors.strip("[").strip("]").split('array')
        vectors = [x for x in vectors if len(x) != 0]
        #print(vectors)
        vectors = get_float_GPT2BERT_vector(vectors)
        float_vecs.append(vectors)
    df_allvec[head] = float_vecs

100%|█████████████████████████████████████████| 480/480 [00:32<00:00, 14.70it/s]


In [5]:
df_cos = pd.DataFrame()
df_euc = pd.DataFrame()
df_euc1 = pd.DataFrame()

for head in heads:
    all_vecs = df_allvec[head].tolist()
    medoidvecs = df_central_vec[head].tolist()
    
    avg_euc = []
    avg_cos = []
    avg_euc = []
    
    for all_vec,medoidvec in tqdm(zip(all_vecs,medoidvecs)):
        medoidvec = np.array(medoidvec)
        
        euc_dis = 0.0
        cos_simi = 0.0
        euc_dis = 0.0
        
        for vec in all_vec:
            vec = np.array(vec)
            cosine_sim = np.dot(vec, medoidvec) / (np.linalg.norm(vec) * np.linalg.norm(medoidvec))
            cos_simi = cos_simi + cosine_sim
            euclidean_dist = np.linalg.norm(vec - medoidvec)
            euc_dis = euc_dis + euclidean_dist
            
        avg_cos_value = cos_simi / (len(all_vecs) - 1)
        avg_cos.append(avg_cos_value)
        
        avg_euc_value = euc_dis / (len(all_vecs) - 1)
        avg_euc.append(avg_euc_value)

    euc1 = (avg_euc - np.mean(avg_euc)) / np.std(avg_euc)
    
    df_cos[head] = avg_cos
    df_euc[head] = avg_euc
    df_euc1[head] = euc1

480it [00:13, 34.60it/s]
480it [00:11, 43.25it/s]
480it [00:09, 49.12it/s]
480it [00:08, 58.11it/s]
480it [00:09, 53.11it/s]
480it [00:10, 47.42it/s]


In [6]:
df_cos.to_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/avg_simi/cos/avg_cos_GPT2_allwords.csv')
df_euc.to_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/avg_simi/euc/avg_euc_GPT2_allwords.csv')
df_euc1.to_csv('/Users/carina/Downloads/courses/final thesis/precessed data/ex2/avg_simi/euc1/avg_euc1_GPT2_allwords.csv')